In [48]:
from calculo_simbolico import compute_tailored_cost, compute_quadratic_cost

In [49]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Grid 10x10 (x = 0..9, y = 0..9)
nx, ny = 10, 10
xs = np.arange(nx)
ys = np.arange(ny)

# Orientations in degrees requested
deg_list = [0, 45, 90, 125]

# Prepare container for log10(cost) maps
maps = []

# Compute cost for each orientation
for deg in deg_list:
    theta = np.deg2rad(deg)
    goal = np.array([0.0, 0.0, theta, 0.0])  # goal pose [x, y, theta, phi]
    cost_map = np.zeros((ny, nx), dtype=float)  # row = y, col = x

    for i, x in enumerate(xs):
        for j, y in enumerate(ys):
            state = np.array([float(x), float(y), 0.0, 0.0])  # sample state
            state_cost, input_cost = compute_tailored_cost(state, goal)
            total = float(state_cost)
            cost_map[ny - 1 - j, i] = total  # flip y so origin (0,0) at bottom-left

    # use log10 for visualization (avoid -inf)
    maps.append(np.log10(cost_map + 1e-12))

# Global min/max for consistent color scale
vmin = min(m.min() for m in maps)
vmax = max(m.max() for m in maps)

# Create 2x2 subplot 3D surfaces
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}]],
    subplot_titles=[f"θ = {d}°" for d in deg_list]
)

positions = [(1,1),(1,2),(2,1),(2,2)]
for m, (r,c), deg in zip(maps, positions, deg_list):
    # meshgrid must match how m was constructed (y was flipped), so use ys[::-1]
    X, Y = np.meshgrid(xs, ys[::-1])
    surf = go.Surface(
        x=X, y=Y, z=m,
        colorscale="Viridis",
        cmin=vmin, cmax=vmax,
        colorbar=dict(title="log10(cost)"),
        showscale=True
    )
    fig.add_trace(surf, row=r, col=c)

    # mark goal (0,0) on each subplot (z value at goal)
    z_goal = m[-1, 0]  # because m was stored with flipped y: row -1 corresponds to y=0
    fig.add_trace(
        go.Scatter3d(x=[0], y=[0], z=[z_goal], mode="markers",
                     marker=dict(color="red", size=4)),
        row=r, col=c
    )

fig.update_layout(height=900, width=1100, title_text="Mapa de Custo 3D (10x10) - Pose alvo = (0,0,θ,0)")
fig.show()

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import Tuple


@dataclass
class TractorTrailerParams:
    """
    Parâmetros geométricos do modelo trator-reboque.

    :param D: Distância do eixo traseiro ao ponto de articulação. (wheelbase do trator)
    :param L: Comprimento do reboque até a quinta roda
    :param epsilon: Parâmetro geométrico relacionado à posição do eixo de articulação.
    """
    D: float = 4.7
    L: float = 6.53
    epsilon: float = 0.736 #nosso caso - negativo


class TractorTrailerGeometry:
    """
    Geometria e métricas homogêneas para um modelo cinemático trator-reboque.

    Esta classe implementa:

    * Cálculo do erro entre ``state`` e ``goal`` no referencial do alvo.
    * Transformação desse erro em coordenadas privilegiadas.
    * Cálculo da distância homogênea (custo de estado) usando pesos não-holonômicos.

    Convenção de estado
    --------------------

    O estado é dado por:

    .. math::

        x = [x, y, \\theta, \\beta]^T

    onde:

    * ``x``: posição longitudinal no mundo
    * ``y``: posição lateral no mundo
    * ``theta``: orientação do trator
    * ``beta``: ângulo de articulação (reboque/trator)

    As entradas (não usadas diretamente aqui) seriam tipicamente:

    .. math::

        u = [v, \\alpha]^T

    com ``v`` velocidade e ``alpha`` ângulo de direção.
    """

    def __init__(self, params: TractorTrailerParams | None = None):
        self.params = params or TractorTrailerParams()
        # Pesos não-holonômicos dos estados (ordem de crescimento)
        # r1 = 1, r2 = 1, r3 = 2, r4 = 3 (exemplo típico)
        self.r: Tuple[int, int, int, int] = (1, 1, 2, 3)
        # Grau de homogeneidade do pseudo-norma (escolha padrão: d = 2 * sum(r))
        self.d: int = 2 * sum(self.r)

    # ------------------------------------------------------------------
    # 1. Erro no referencial do alvo
    # ------------------------------------------------------------------
    def _relative_error(self, state: np.ndarray, goal: np.ndarray) -> Tuple[float, float, float, float]:
        """
        Calcula o erro entre ``state`` e ``goal`` no frame do alvo.

        Parameters
        ----------
        state : np.ndarray
            Estado atual, ``[x, y, theta, beta]``.
        goal : np.ndarray
            Estado alvo, ``[x_g, y_g, theta_g, beta_g]``.

        Returns
        -------
        (x_rel, y_rel, theta_rel, beta_rel) : tuple of float
            Erros nas coordenadas do frame do alvo.

        Notas
        -----
        1. Transladamos o estado atual para o sistema com origem no goal:

           .. math::

               \\Delta x = x - x_g, \\quad \\Delta y = y - y_g

        2. Rotacionamos (\\Delta x, \\Delta y) pelo ângulo do alvo
           para obter o erro no frame do alvo:

           .. math::

               \\begin{bmatrix}
                   x_\\text{rel} \\\\
                   y_\\text{rel}
               \\end{bmatrix}
               =
               R(-\\theta_g)
               \\begin{bmatrix}
                   \\Delta x \\\\
                   \\Delta y
               \\end{bmatrix}

        3. Os erros de orientação e articulação são:

           .. math::

               \\theta_\\text{rel} = \\theta - \\theta_g, \\quad
               \\beta_\\text{rel} = \\beta - \\beta_g
        """
        x, y, th, be = state
        xg, yg, thg, beg = goal

        dx = x - xg
        dy = y - yg
        c = float(np.cos(thg))
        s = float(np.sin(thg))

        # Rotação para o frame do alvo
        x_rel = c * dx + s * dy
        y_rel = -s * dx + c * dy

        th_rel = th - thg
        be_rel = be - beg

        return float(x_rel), float(y_rel), float(th_rel), float(be_rel)

    # ------------------------------------------------------------------
    # 2. Coordenadas privilegiadas z(state, goal)
    # ------------------------------------------------------------------
    def privileged_coords(self, state: np.ndarray, goal: np.ndarray) -> np.ndarray:
        """
        Calcula as coordenadas privilegiadas do erro entre dois estados.

        Parameters
        ----------
        state : np.ndarray
            Estado atual, ``[x, y, theta, beta]``.
        goal : np.ndarray
            Estado alvo, ``[x_g, y_g, theta_g, beta_g]``.

        Returns
        -------
        z : np.ndarray
            Vetor de coordenadas privilegiadas ``[z1, z2, z3, z4]``.

        Conceito
        --------
        As coordenadas privilegiadas são um sistema de coordenadas
        adaptado à estrutura não-holonômica do sistema. Para o modelo
        trator-reboque simples, usamos uma forma inspirada na literatura
        e em exemplos de carros não-holonômicos:

        1. Primeiro calculamos o erro relativo:

           .. math::

               (x_\\text{rel}, y_\\text{rel}, \\theta_\\text{rel}, \\beta_\\text{rel})

        2. Em seguida, definimos:

           .. math::

               z_1 &= x_\\text{rel} \\\\
               z_2 &= L \\beta_\\text{rel} - \\theta_\\text{rel} \\\\
               z_3 &= y_\\text{rel} - L^2 \\beta_\\text{rel} \\\\
               z_4 &= \\frac{L^2 \\beta_\\text{rel}}{\\varepsilon - 1}

        onde ``L`` e ``epsilon`` vêm dos parâmetros geométricos.
        """
        L = self.params.L
        eps = self.params.epsilon

        x_rel, y_rel, th_rel, be_rel = self._relative_error(state, goal)

        z1 = x_rel
        z2 = L * be_rel - th_rel
        z3 = y_rel - (L ** 2) * be_rel
        # Evitar divisão por zero se epsilon for 1 (caso degenerado)
        denom = eps - 1.0
        if abs(denom) < 1e-9:
            raise ValueError("epsilon muito próximo de 1: coordenada z4 torna-se singular.")
        z4 = (L ** 2) * be_rel / denom

        return np.array([z1, z2, z3, z4], dtype=float)

    # ------------------------------------------------------------------
    # 3. Distância homogênea (custo de estado)
    # ------------------------------------------------------------------
    def homogeneous_distance(
        self,
        state: np.ndarray,
        goal: np.ndarray,
        q: Tuple[float, float, float, float] = (1.0, 1.0, 1.0, 1.0),
    ) -> float:
        """
        Calcula a distância homogênea (custo) entre dois estados.

        A métrica utiliza coordenadas privilegiadas e pesos não-holonômicos,
        penalizando erros em direções mais difíceis com potências adequadas.

        Parameters
        ----------
        state : np.ndarray
            Estado atual, ``[x, y, theta, beta]``.
        goal : np.ndarray
            Estado alvo, ``[x_g, y_g, theta_g, beta_g]``.
        q : tuple of float, optional
            Pesos dos estados no custo, ``(q1, q2, q3, q4)``.

        Returns
        -------
        cost : float
            Valor escalar da distância homogênea.

        Fórmula
        -------
        Seja :math:`z = (z_1, z_2, z_3, z_4)` o vetor de coordenadas
        privilegiadas e :math:`r = (r_1, r_2, r_3, r_4)` os pesos
        não-holonômicos (ordens). Definimos o grau de homogeneidade ``d`` e:

        .. math::

            \\ell(z) = \\sum_{i=1}^4 q_i \\; |z_i|^{d / r_i}

        onde tipicamente escolhemos :math:`d = 2 \\sum_i r_i`.

        Intuição
        --------
        * Direções mais fáceis (menor :math:`r_i`) recebem expoentes maiores.
        * Direções mais difíceis (maior :math:`r_i`) recebem expoentes menores,
          o que as torna relativamente mais caras perto da origem.
        """
        z = self.privileged_coords(state, goal)
        r = self.r
        d = float(self.d)

        cost = 0.0
        for i in range(4):
            exp = d / r[i]
            cost += q[i] * (abs(z[i]) ** exp)
        return float(cost)


# ----------------------------------------------------------------------
# Exemplo rápido de uso
# ----------------------------------------------------------------------
if __name__ == "__main__":
    params = TractorTrailerParams(D=4.7, L=6.53, epsilon=0.736)
    geom = TractorTrailerGeometry(params)

    state = np.array([0.0, 0.0, 0.0, 5.00])
    goal = np.array([0.0, 0.0, 0.0, 0.0])

    z = geom.privileged_coords(state, goal)
    dist = geom.homogeneous_distance(state, goal)

    print("z =", z)
    print("homogeneous distance =", dist)


z = [   0.           32.65       -213.2045     -807.59280303]
homogeneous distance = 1.5644805480816745e+21
